In [149]:
# %pip install tensorflow
# %pip freeze > requirements.txt

In [150]:
# generate training set
from tensorflow import keras

train_set = keras.preprocessing.image_dataset_from_directory(
	# '../../input/images/training_testing/random-inf-0.8/training/',
	'../../input/images/training_testing/random-1000-0.8/training/',
	validation_split=0.2,
	subset="training",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)
train_set.class_names


Found 2000 files belonging to 2 classes.
Using 1600 files for training.


['no whale', 'whale present']

In [151]:
# generate validation set
validation_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/training_testing/random-10a00-0.8/training/',
	validation_split=0.2,
	subset="validation",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)

Found 400 files belonging to 2 classes.
Using 80 files for validation.


In [152]:
for images, labels in train_set:
    preprocessed_images = keras.applications.resnet50.preprocess_input(images)

preprocessed_images, labels


(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
 array([[[[ 16.060997 , -60.779    ,  62.32     ],
          [ 16.060997 , -60.779    ,  62.32     ],
          [ 18.060997 ,  61.221    , 130.32     ],
          ...,
          [ 15.060997 ,  57.221    , 130.32     ],
          [ 10.060997 ,  50.221    , 130.32     ],
          [ 10.060997 ,  50.221    , 130.32     ]],
 
         [[ 16.060997 , -60.779    ,  62.32     ],
          [ 16.060997 , -60.779    ,  62.32     ],
          [ 18.060997 ,  61.221    , 130.32     ],
          ...,
          [ 15.060997 ,  57.221    , 130.32     ],
          [ 10.060997 ,  50.221    , 130.32     ],
          [ 10.060997 ,  50.221    , 130.32     ]],
 
         [[ 16.060997 , -60.779    ,  62.32     ],
          [ 16.060997 , -60.779    ,  62.32     ],
          [ 18.060997 ,  61.221    , 130.32     ],
          ...,
          [ 15.060997 ,  57.221    , 130.32     ],
          [ 10.060997 ,  50.221    , 130.32     ],
          [ 10.060997 

In [153]:
# load resnet50 model with pretrained weights
# replace output layer with 2 neurons
# connect to gpu?
# shuffle training set
#  cross entrpoy loss, otimize adam model parameters learning rate 0.0001 epochs 10
# save model
# confusion matrix

In [154]:
# base_model = keras.applications.ResNet50(
# 	weights= None, 
# 	# include_top=False, 
# 	# input_shape= (img_height,img_width,3)
# 	num_classes=2
# 	)

imported_model= keras.applications.ResNet50(include_top=False,
	input_shape=(224,224,3),
	pooling='avg',
	classes=2,
	weights='imagenet'
	# weights=None
)

In [155]:
custom_model = keras.models.Sequential()
custom_model.add(imported_model)
custom_model.add(keras.layers.Dense(2, activation='softmax'))
custom_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_10 (Dense)            (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 23538690 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [156]:
from keras.optimizers.legacy import Adam
custom_model.compile(
	optimizer=Adam(learning_rate=0.001),
	loss='sparse_categorical_crossentropy',
	# loss='categorical_crossentropy',
	metrics=['accuracy']
)

In [157]:
train_set.shuffle(1000)
history = custom_model.fit(
	train_set,
	validation_data=validation_set,
	epochs=5
)

Epoch 1/5
50/50 [==============================] - 132s 3s/step - loss: 0.5751 - accuracy: 0.8188 - val_loss: 213087.4062 - val_accuracy: 0.4750
Epoch 2/5
50/50 [==============================] - 131s 3s/step - loss: 0.3443 - accuracy: 0.8606 - val_loss: 1.8661 - val_accuracy: 0.6250
Epoch 3/5
50/50 [==============================] - 127s 3s/step - loss: 0.2467 - accuracy: 0.8975 - val_loss: 1.1949 - val_accuracy: 0.5750
Epoch 4/5
50/50 [==============================] - 126s 3s/step - loss: 0.1848 - accuracy: 0.9200 - val_loss: 0.3906 - val_accuracy: 0.7875
Epoch 5/5
50/50 [==============================] - 119s 2s/step - loss: 0.1259 - accuracy: 0.9456 - val_loss: 0.3882 - val_accuracy: 0.8625


In [164]:
# evaluation = custom_model.evaluate(validation_set)
evaluation = custom_model.evaluate(train_set)
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])


79/79 [==============================] - 47s 590ms/step - loss: 0.3792 - accuracy: 0.8457
Validation Loss: 0.37920284271240234
Validation Accuracy: 0.8457256555557251


In [159]:
# save custom_model
custom_model.save('resnet50_epochs5_samples1000_seed123_entr.h5')

# for ten epochos of all the data it takes roughly 5 hours of work

# resnet50_epochs3_samples2000_seed123_entr takes 20 minutes and gets 90% accuracy

/Users/lshuster/Library/CloudStorage/OneDrive-GlobalHealthcareExchange/Desktop/whale-detection/.conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [163]:
total_set = keras.preprocessing.image_dataset_from_directory(
	# '../../input/images/training_testing/random-inf-0.8/training/',
	'../../input/images/training_testing/random-1000-0.8/training/',
	# validation_split=0.2,
	# subset="training",
	# seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)
train_set.class_names

Found 2515 files belonging to 2 classes.


['no whale', 'whale present']